<a href="https://colab.research.google.com/github/hseyeon1006/ESAA_YB/blob/main/XGBoost2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

# os 관리를 위한 라이브러리
import os
import gc

In [2]:
# 경고메세지 끄기

import warnings
warnings.filterwarnings(action = 'ignore')

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [4]:
seed_everything(42) # Seed 고정

In [5]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# 전처리 파일 불러오기
train = pd.read_parquet('/content/drive/MyDrive/Colab_Notebooks/학회/방학 프로젝트/train_pre.parquet')
test = pd.read_parquet('/content/drive/MyDrive/Colab_Notebooks/학회/방학 프로젝트/test_pre.parquet')
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/학회/방학 프로젝트/sample_submission.csv', index_col = 0)

In [12]:
train.head(3)

,Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Delay,Holiday
1,8,460,624,13930,14869,1250.0,20304,310,NaN,0
4,1,540,619,14771,10157,250.0,20304,554,NaN,0
5,4,945,1011,11618,11278,199.0,20452,3430,1.0,0


In [13]:
test.head(3)

,Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Holiday
0,12,716,779,12266,14683,191.0,19977,4382,0
1,9,900,1035,11618,10397,746.0,19790,1934,0
2,3,960,1155,13930,12953,733.0,19977,2144,0


# **3. 준지도학습**
- 라벨이 없는 데이터에 대한 라벨링 진행

## **3-0. 데이터 분리**

In [14]:
### 데이터 분리
# 레이블이 있는 데이터와 없는 데이터 분리

train_labeled = train.loc[train['Delay'].notnull(),:]
train_unlabeled = train.loc[train['Delay'].isnull(),:]

In [15]:
train_labeled.shape[0]

172245

In [16]:
train_unlabeled.shape[0]

502770

- 라벨링 된 데이터에 비해 라벨링 되지 **않은** 데이터의 수가 더 많다.

In [17]:
### X,y 분리

# train
X_labeled = train_labeled.drop(['Delay'],axis = 1)
y_labeled = train_labeled['Delay']
X_unlabeled = train_unlabeled.drop(['Delay'],axis = 1)

print("X Labeled: ", X_labeled.shape)
print("y Labeled: ", y_labeled.shape)
print()
print("X Unlabeled: ", X_unlabeled.shape)

# test
X_test = test
print()
print("X Test: ", X_test.shape)

X Labeled:  (172245, 9)
y Labeled:  (172245,)

X Unlabeled:  (502770, 9)

X Test:  (1000000, 9)


## **3-1. 기본 모델 모델링(Base Model Modeling)**
- Base Model 선정 & 학습
  - 레이블이 **있는** 데이터만을 사용하여 모델을 학습시키면서 튜닝을 진행
- 기본 분류기인 Base Model의 하이퍼 파라미터 튜닝을 수행
  - ```GridSearchCV```, ```RandomizedSearchCV```, 또는 ```Bayesian optimization``` 등의 방법 활용
  - 주어진 데이터와 문제에 맞는 하이퍼파라미터 조합을 찾는 것이 중요

### **a) 초기 모델링**
- ```GBM``` 모델링
- 학습
  - 교차 검증 방식 활용
  - y_labeled에서 0(= Not Delayed)과 1(= Delayed)의 분포에 차이가 있음
  - y의 분포를 고려한 교차 검증 수행을 위해 ```StratifiedKFold``` 활용
- 평가 지표: LogLoss



**📌 LogLoss**  
- $- (1/N) * Σ[y_i * log(p_i) + (1 - y_i) * log(1 - p_i)]$
  - N: 데이터 포인트(샘플)의 수
  - y_i: 실제 레이블(이진 분류의 경우 0 또는 1)
  - p_i
    - 모델의 예측값 => 해당 데이터 포인트가 1(= Delayed)에 속할 확률
    - 일반적으로 모델의 ```predict_proba()``` 메서드를 사용하여 얻음
- 각 데이터 포인트에서 실제 레이블과 모델의 예측값 사이의 로그 손실을 계산하고, 이를 모든 데이터 포인트에 대해 평균하여 구함
  - 작을수록 good

In [20]:
import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_predict, StratifiedKFold

### 모델 객체 생성
xgb_labeled = XGBClassifier(random_state = 42)

### 학습 & 예측
# StratifiedKFold를 사용하여 5-Fold 교차 검증을 수행
n_folds = 5
skf = StratifiedKFold(n_splits = n_folds, shuffle = True, random_state = 42)

# 교차 검증을 통해 예측값 구하기
y_labeled_pred = cross_val_predict(xgb_labeled, X_labeled, y_labeled,
                           cv = skf, method = 'predict_proba')

### 평가
# logloss를 계산하기 위해 실제 레이블과 예측값의 logloss 계산
print('LogLoss: {0:.4f}'.format(log_loss(y_labeled, y_labeled_pred)))

LogLoss: 0.4436


### **b) 하이퍼 파라미터 튜닝**
- 성능을 높여봅시다 유후 ^ㅇ^
- ```GridSearchCV``` 활용

In [21]:
from sklearn.model_selection import GridSearchCV

### 파라미터 목록
param_grid = {'max_depth':[3,4,5,6],
        'n_estimators':[100,200],
        'learning_rate':[0.01,0.1],
        }

### 최적의 하이퍼파라미터 탐색
grid_cv = GridSearchCV(estimator = xgb_labeled, param_grid = param_grid,
                       cv = skf, scoring = 'neg_log_loss', verbose = 0)
grid_cv.fit(X_labeled, y_labeled)

### 각 조합별로 LogLoss 출력
results = grid_cv.cv_results_
for mean_score, params in zip(results["mean_test_score"], results["params"]):
    print(f"하이퍼파라미터 조합: {params}, 평균 Log Loss: {abs(mean_score)}")

하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}, 평균 Log Loss: 0.49240976782274054
하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200}, 평균 Log Loss: 0.4558882106816754
하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 100}, 평균 Log Loss: 0.4913710265856036
하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 200}, 평균 Log Loss: 0.4544580468841236
하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100}, 평균 Log Loss: 0.4906545902105842
하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 200}, 평균 Log Loss: 0.4535242735374859
하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 100}, 평균 Log Loss: 0.48991271997899516
하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 200}, 평균 Log Loss: 0.45255665395722494
하이퍼파라미터 조합: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}, 평균 Log Loss: 0.44280270290406004
하이퍼파라미터 조합: {'learning_rate': 0.1, 'max_dep

**📌 왜 ```scoring = 'neg_log_loss'```?**  
- Scikit-learn에서 사용하는 scoring 파라미터는 기본적으로 양의 값이 좋은 평가 지표를 의미
  - LogLoss와 같이 작을수록 좋은 평가 지표인 경우에는 (-) 기호를 붙여 음수값을 사용

최적의 파라미터 : learning_rate = 0.1, max_depth = 5, n_estimators = 200

In [22]:
### 최적 파라미터로 튜닝된 모델을 base model로 생성

base_model = XGBClassifier(learning_rate = 0.1, max_depth = 5, n_estimators = 200)

## **3-2. Self Training**
- 초기에 레이블이 없는 데이터를 레이블링하여 일부 레이블이 있는 데이터를 생성한 후, 이 데이터를 활용해 모델을 학습시키는 방법
- 학습된 모델을 사용하여 레이블이 없는 데이터에 대한 예측을 수행하고, 예측 결과 중 신뢰할 수 있는 일부 데이터를 레이블링에 추가로 사용하여 새로운 학습용 데이터를 생성
> 해당 과정을 반복하여 모델을 업데이트하면서 점진적으로 더 많은 레이블을 확보해나가는 방식

In [23]:
from sklearn.semi_supervised import SelfTrainingClassifier

# Self Training 분류기 생성
self_training_model = SelfTrainingClassifier(base_model)

In [24]:
def run_self_training(self_training_model, X_labeled, y_labeled, X_unlabeled, early_stopping_rounds=5):
    iter = 0  # 반복 횟수 카운트를 위한 변수
    rounds_without_improvement = 0  # early stopping을 위한 카운트 변수

    while X_unlabeled.shape[0] > 0 and rounds_without_improvement < early_stopping_rounds:
        iter = iter + 1
        print("=== Iteration {} ===".format(iter))

        ## 1. 레이블이 있는(labeled) 데이터로 모델 학습
        self_training_model.fit(X_labeled, y_labeled)

        ## 2. 훈련된 모델로 레이블이 없는 데이터를 분류
        y_pred_unlabeled = self_training_model.predict(X_unlabeled)

        ## 3. 의사 레이블(pseudo label) 설정
        # 모델이 90% 이상 확신하는 데이터를 labeled 데이터에 추가
        confident_mask = self_training_model.predict_proba(X_unlabeled).max(axis=1) > 0.9
        X_confident = X_unlabeled[confident_mask]
        y_confident = y_pred_unlabeled[confident_mask]
        # 신뢰할 수 있는 데이터를 레이블링에 추가하여 새로운 학습용 데이터 구성
        # 레이블링 된 데이터의 수를 증가시키는 과정
        X_labeled = np.concatenate([X_labeled, X_confident], axis=0)
        y_labeled = np.concatenate([y_labeled, y_confident], axis=0)
        # 의사 레이블링 된 데이터들은 이제 레이블이 없는 데이터에서 제거
        X_unlabeled = X_unlabeled[~confident_mask]

        print("{}개 레이블링 완료".format(X_confident.shape[0]))

        ## Early Stopping 체크
        if X_confident.shape[0] <= 30:
            rounds_without_improvement += 1
            print("Early Stopping 카운트: {}/{}".format(rounds_without_improvement, early_stopping_rounds))
            print()
        else:
            rounds_without_improvement = 0
            print()

    print()
    print("레이블링 완료")

    # 라벨링 된 데이터 반환
    return X_labeled, y_labeled, X_unlabeled

In [25]:
# Self-training을 통한 레이블링 수행

X_labeled, y_labeled, X_unlabeled = run_self_training(base_model, X_labeled, y_labeled, X_unlabeled,
                                                      early_stopping_rounds = 10)

=== Iteration 1 ===
79501개 레이블링 완료

=== Iteration 2 ===
56356개 레이블링 완료

=== Iteration 3 ===
42209개 레이블링 완료

=== Iteration 4 ===
28332개 레이블링 완료

=== Iteration 5 ===
20483개 레이블링 완료

=== Iteration 6 ===
16861개 레이블링 완료

=== Iteration 7 ===
15244개 레이블링 완료

=== Iteration 8 ===
12656개 레이블링 완료

=== Iteration 9 ===
9271개 레이블링 완료

=== Iteration 10 ===
7005개 레이블링 완료

=== Iteration 11 ===
6605개 레이블링 완료

=== Iteration 12 ===
6739개 레이블링 완료

=== Iteration 13 ===
5462개 레이블링 완료

=== Iteration 14 ===
4857개 레이블링 완료

=== Iteration 15 ===
3329개 레이블링 완료

=== Iteration 16 ===
2397개 레이블링 완료

=== Iteration 17 ===
1276개 레이블링 완료

=== Iteration 18 ===
1252개 레이블링 완료

=== Iteration 19 ===
1125개 레이블링 완료

=== Iteration 20 ===
1327개 레이블링 완료

=== Iteration 21 ===
1328개 레이블링 완료

=== Iteration 22 ===
1487개 레이블링 완료

=== Iteration 23 ===
1846개 레이블링 완료

=== Iteration 24 ===
1881개 레이블링 완료

=== Iteration 25 ===
1765개 레이블링 완료

=== Iteration 26 ===
1283개 레이블링 완료

=== Iteration 27 ===
1367개 레이블링 완료

=== Iteration 28 ===
1425개 레이

In [26]:
print(X_labeled.shape[0])
print(X_unlabeled.shape[0]/(X_labeled.shape[0] + X_unlabeled.shape[0]))

531723
0.2122797271171752


- 라벨링 된 데이터의 개수가 172245에서 531723로 증가
- 아직 약 21%의 데이터가 라벨링되지 않은 상황

In [27]:
### 최종 데이터 병합
# 이미 데이터 수는 충분히 많은 것 같기에 그냥 날리고 가는걸로..

X_train_final = X_labeled
y_train_final = y_labeled

# **4. 최종 예측을 위한 모델링**

### **a) 초기 모델링**

In [28]:
### 모델 객체 생성
xgb_ssl = XGBClassifier(random_state = 42)

### 학습 & 예측
# StratifiedKFold를 사용하여 5-Fold 교차 검증을 수행
n_folds = 5
skf = StratifiedKFold(n_splits = n_folds, shuffle = True, random_state = 42)
# 교차 검증을 통해 예측값 구하기
y_ssl_pred = cross_val_predict(xgb_ssl, X_train_final, y_train_final,
                           cv = skf, method = 'predict_proba')

### 평가
# logloss를 계산하기 위해 실제 레이블과 예측값의 logloss 계산
print('LogLoss: {0:.4f}'.format(log_loss(y_train_final, y_ssl_pred)))

LogLoss: 0.1924


### **b) 하이퍼 파라미터 튜닝**

In [29]:
from sklearn.model_selection import GridSearchCV

### 파라미터 목록
param_grid_final = {'max_depth':[3,4,5,6],
        'n_estimators':[100,200],
        'learning_rate':[0.01,0.1],
        }

### 최적의 하이퍼파라미터 탐색
grid_cv_final = GridSearchCV(estimator = xgb_ssl, param_grid = param_grid_final,
                       cv = skf, scoring = 'neg_log_loss')
grid_cv_final.fit(X_train_final, y_train_final)

### 각 조합별로 LogLoss 출력
results2 = grid_cv_final.cv_results_
for mean_score, params in zip(results2["mean_test_score"], results2["params"]):
    print(f"하이퍼파라미터 조합: {params}, 평균 Log Loss: {abs(mean_score)}")

print("최적 하이퍼 파라미터:\n", grid_cv_final.best_params_)
print("최적 LogLoss: {0:4f}".format(-grid_cv_final.best_score_))

하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}, 평균 Log Loss: 0.3136965147571863
하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200}, 평균 Log Loss: 0.227443393087681
하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 100}, 평균 Log Loss: 0.3120411157451202
하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 200}, 평균 Log Loss: 0.22487442745508074
하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100}, 평균 Log Loss: 0.3107950170640562
하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 200}, 평균 Log Loss: 0.22317039876917577
하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 100}, 평균 Log Loss: 0.3099968530638692
하이퍼파라미터 조합: {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 200}, 평균 Log Loss: 0.22170749294377617
하이퍼파라미터 조합: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}, 평균 Log Loss: 0.1954411647722489
하이퍼파라미터 조합: {'learning_rate': 0.1, 'max_depth

### **c) 최종 예측**

- Not_Delayed = 0
- Delayed = 1

In [30]:
final_classifier = XGBClassifier(learning_rate = 0.1, max_depth = 6, n_estimators = 200)
final_classifier.fit(X_train_final, y_train_final)
y_test_pred = final_classifier.predict_proba(X_test)

In [31]:
y_test_pred

array([[0.03732264, 0.96267736],
       [0.06566709, 0.9343329 ],
       [0.25257355, 0.74742645],
       ...,
       [0.26889265, 0.73110735],
       [0.0809536 , 0.9190464 ],
       [0.04063028, 0.9593697 ]], dtype=float32)

# **5. 최종 제출 파일 생성**

In [32]:
sample_submission.head(3)

,Not_Delayed,Delayed
ID,,
TEST_000000,0,1
TEST_000001,0,1
TEST_000002,0,1


In [33]:
submission = pd.DataFrame(data = y_test_pred, columns = sample_submission.columns,
                          index = sample_submission.index)

In [34]:
submission.head(3)

,Not_Delayed,Delayed
ID,,
TEST_000000,0.037323,0.962677
TEST_000001,0.065667,0.934333
TEST_000002,0.252574,0.747426


In [35]:
submission.to_csv('xgb_submission.csv', index = True)

## 리더보드 제출 결과
- public: 1.0422
- private: 1.6564
- 150 ~ 160위 정도
- 개선방안?
    - 직접 하이퍼파라미터 튜닝을 하는 대신 Pycaret, Optuna 등의 AutoML 기법 활용
    - 그러나 GBM 너ㅓㅓㅓㅓ무 오래 걸려서 다른 모델로 시도해 보는 현이 좋을 듯함